# Importando as Bibliotecas e Pacotes

Essa é uma competição da Kaggle em que tem a proposta de treinar as habilidades de ML(Machine Learning)

As respostas serão medidas por meio da **ROC curve** que calculará a área abaixo do gráfico o score será a diferença entre a predição e probabilidade por meio do site:https://www.kaggle.com/c/tabular-playground-series-nov-2021/overview/description 

In [21]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau 
from tensorflow.keras import Sequential, Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dense, Flatten, InputLayer, Dropout, Input
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score, accuracy_score

from warnings import filterwarnings
filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
sub_data = pd.read_csv('sample_submission.csv')

# Entendendo os dados de teste e treino

## Dados de treinamento

In [4]:
train.head()

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,target
0,0,0.106643,3.59437,132.8040,3.18428,0.081971,1.18859,3.73238,2.266270,2.09959,...,1.09862,0.013331,-0.011715,0.052759,0.065400,4.211250,1.97877,0.085974,0.240496,0
1,1,0.125021,1.67336,76.5336,3.37825,0.099400,5.09366,1.27562,-0.471318,4.54594,...,3.46017,0.017054,0.124863,0.154064,0.606848,-0.267928,2.57786,-0.020877,0.024719,0
2,2,0.036330,1.49747,233.5460,2.19435,0.026914,3.12694,5.05687,3.849460,1.80187,...,4.88300,0.085222,0.032396,0.116092,-0.001689,-0.520069,2.14112,0.124464,0.148209,0
3,3,-0.014077,0.24600,779.9670,1.89064,0.006948,1.53112,2.69800,4.517330,4.50332,...,3.47439,-0.017103,-0.008100,0.062013,0.041193,0.511657,1.96860,0.040017,0.044873,0
4,4,-0.003259,3.71542,156.1280,2.14772,0.018284,2.09859,4.15492,-0.038236,3.37145,...,1.91059,-0.042943,0.105616,0.125072,0.037509,1.043790,1.07481,-0.012819,0.072798,1


In [5]:
train.describe()

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,target
count,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,...,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000,600000.000000
mean,299999.500000,0.306508,2.497590,306.644536,2.647901,0.177850,2.556832,2.699650,2.571593,2.538273,...,2.444471,0.155260,0.059407,0.144932,0.106419,2.547853,2.590159,0.158881,0.123048,0.506010
std,173205.225094,0.522450,1.554018,551.743893,1.544529,0.417488,1.562527,1.564000,1.549361,1.532988,...,1.542509,0.548397,0.119426,0.462015,0.209128,1.558427,1.525091,0.436190,0.264896,0.499964
min,0.000000,-3.797450,-1.223960,-1842.530000,-1.368560,-3.206210,-1.169770,-1.059310,-1.281970,-1.242020,...,-1.217700,-9.761770,-4.666240,-3.101500,-1.276540,-1.584740,-1.254730,-3.993500,-2.783380,0.000000
25%,149999.750000,0.026222,1.186238,43.573400,1.442028,0.019709,1.261038,1.385820,1.333848,1.292163,...,1.214178,0.018904,0.024483,0.017055,0.025461,1.247888,1.348078,0.013536,0.018105,0.000000
50%,299999.500000,0.097788,2.516500,133.626000,2.634130,0.061586,2.590425,2.801255,2.557985,2.475880,...,2.386845,0.068906,0.056649,0.063439,0.062151,2.601940,2.682090,0.058058,0.058471,1.000000
75%,449999.250000,0.397184,3.787630,302.262250,3.907640,0.112712,3.813662,3.996912,3.823450,3.804360,...,3.693872,0.125165,0.088162,0.113114,0.102016,3.820665,3.839520,0.110718,0.104872,1.000000
max,599999.000000,8.781500,6.226720,6119.280000,6.521150,8.265470,6.515070,6.586780,6.258770,6.389670,...,6.573890,18.412800,10.211800,8.623270,3.657220,6.254360,6.145300,10.767000,5.988110,1.000000


In [12]:
train[train.isnull()].sum().sort_values(ascending = False)

target    0.0
f36       0.0
f26       0.0
f27       0.0
f28       0.0
         ... 
f69       0.0
f70       0.0
f71       0.0
f72       0.0
id        0.0
Length: 102, dtype: float64

In [13]:
train.shape

(600000, 102)

### Como podemos ver os dados de treino não possui valores vazios e possui uma estrutura:

In [15]:
print(f'A base de dados de treino possui {train.shape[0]} linhas e {train.shape[1]} colunas')

A base de dados de treino possui 600000 linhas e 102 colunas


## Dados de teste

In [16]:
test.head()

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
0,600000,0.003229,4.838660,585.529,2.282910,0.713180,3.907830,0.480696,1.482270,4.891810,...,0.112910,1.073550,0.122149,0.086330,0.036010,0.010619,0.290343,1.898200,0.131533,0.012047
1,600001,0.008602,0.505536,-100.099,3.012670,0.027199,1.194610,5.036620,2.517440,4.553890,...,-0.020214,2.622340,0.123307,0.033063,0.123059,0.005771,-0.392923,3.689640,0.047418,0.120015
2,600002,1.461000,2.437260,-112.964,3.541230,0.752338,4.338310,1.648080,4.699910,1.950250,...,-0.011036,2.030180,-0.000426,0.084091,0.123605,0.499554,4.054650,3.330670,0.108843,0.064687
3,600003,0.140556,3.085610,179.451,0.573945,0.057342,2.216790,1.623480,0.526174,1.542540,...,0.050117,0.221613,0.045298,0.129966,0.004015,0.018279,2.696580,-0.533491,0.052524,0.011058
4,600004,0.128876,5.199760,107.466,-0.497149,0.080220,0.458121,0.629839,5.240460,-0.232279,...,0.058860,2.660430,0.135425,0.036481,0.093912,0.056315,1.110710,3.584470,0.145319,-0.050393


In [17]:
test.describe()

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
count,5.400000e+05,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,...,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000
mean,8.699995e+05,0.348663,2.618251,263.577730,2.583735,0.211793,2.709543,2.762370,2.479384,2.549483,...,0.072670,2.600771,0.154676,0.056792,0.124517,0.092847,2.442579,2.640276,0.158335,0.140386
std,1.558847e+05,0.566251,1.543507,496.444309,1.529646,0.468154,1.568074,1.586975,1.520741,1.520833,...,0.095479,1.531937,0.546308,0.135927,0.420197,0.191741,1.567684,1.537125,0.430856,0.292759
min,6.000000e+05,-3.628650,-1.260150,-1764.160000,-1.370560,-3.868460,-1.180650,-1.126370,-1.284250,-1.084660,...,-0.610129,-1.438570,-8.970460,-5.565240,-2.958050,-1.907490,-1.939050,-1.232050,-4.537930,-2.305220
25%,7.349998e+05,0.043555,1.326280,27.361175,1.408300,0.028489,1.400490,1.433132,1.278905,1.306570,...,0.025997,1.368090,0.019573,0.019760,0.020277,0.020370,1.117510,1.385690,0.018050,0.021656
50%,8.699995e+05,0.115869,2.657140,115.631000,2.547380,0.072008,2.797510,2.892945,2.456350,2.485895,...,0.059437,2.576965,0.067471,0.051755,0.064928,0.056317,2.458465,2.740695,0.063792,0.061905
75%,1.004999e+06,0.457939,3.884850,245.446250,3.812990,0.125637,3.982830,4.083213,3.694180,3.811365,...,0.094415,3.859483,0.122299,0.083540,0.110353,0.095058,3.737155,3.903460,0.115588,0.108654
max,1.139999e+06,8.666950,6.434070,6098.190000,6.275570,7.915200,6.444200,6.726750,6.339400,6.180310,...,1.966860,6.380110,19.249500,11.484900,8.493580,4.097870,6.210380,6.421930,10.777000,5.414280


In [18]:
test[test.isnull()].sum().sort_values(ascending = False)

f99    0.0
f35    0.0
f25    0.0
f26    0.0
f27    0.0
      ... 
f68    0.0
f69    0.0
f70    0.0
f71    0.0
id     0.0
Length: 101, dtype: float64

### Como podemos ver os dados de teste não possui valores vazios e possui uma estrutura:

In [19]:
print(f'A base de dados de teste possui {test.shape[0]} linhas e {test.shape[1]} colunas')

A base de dados de teste possui 540000 linhas e 101 colunas


# Preparando os dados teste e treino para o modelo

In [26]:
# Separar treino em X e y para o código do modelo e X_test para teste
X = train.drop(columns=['id','target']).copy()
y = train['target'].copy()
X_test = test.drop(columns='id').copy()

### Por conta dos dados estarem fora de padrão, temos que utilizar o Standard Scaler para deixa-los em uma mesmoa escala 

In [27]:
scaler = StandardScaler()
X = pd.DataFrame(columns=X.columns, data=scaler.fit_transform(X))
X_test = pd.DataFrame(columns=X_test.columns, data=scaler.transform(X_test))

In [29]:
X.describe()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
count,6.000000e+05,6.000000e+05,6.000000e+05,6.000000e+05,6.000000e+05,6.000000e+05,6.000000e+05,6.000000e+05,6.000000e+05,6.000000e+05,...,6.000000e+05,6.000000e+05,6.000000e+05,6.000000e+05,6.000000e+05,6.000000e+05,6.000000e+05,6.000000e+05,6.000000e+05,6.000000e+05
mean,8.930625e-17,3.231434e-16,-7.223678e-16,8.729655e-16,-8.712725e-16,-2.956810e-15,-3.699074e-16,-3.743796e-16,1.984207e-16,2.502933e-17,...,1.018022e-15,1.792962e-16,2.429393e-16,1.004084e-16,-2.785957e-17,-6.900434e-17,-3.883360e-16,-6.855715e-16,-1.417276e-16,3.396825e-16
std,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,...,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,-7.855227e+00,-2.394794e+00,-3.895243e+00,-2.600446e+00,-8.105763e+00,-2.384985e+00,-2.403430e+00,-2.487197e+00,-2.465967e+00,-6.428692e+00,...,-3.327214e+01,-2.374167e+00,-1.808369e+01,-3.956981e+01,-7.026692e+00,-6.612998e+00,-2.651774e+00,-2.521090e+00,-9.519664e+00,-1.097197e+01
25%,-5.364845e-01,-8.438471e-01,-4.767998e-01,-7.807393e-01,-3.787915e-01,-8.292945e-01,-8.400459e-01,-7.988755e-01,-8.128648e-01,-2.721238e-01,...,-4.505446e-01,-7.975928e-01,-2.486452e-01,-2.924307e-01,-2.767824e-01,-3.871222e-01,-8.341527e-01,-8.144316e-01,-3.332156e-01,-3.961681e-01
50%,-3.995015e-01,1.216877e-02,-3.135851e-01,-8.916106e-03,-2.784834e-01,2.149922e-02,6.496472e-02,-8.783022e-03,-4.070035e-02,-1.792356e-01,...,-1.482936e-01,-3.735838e-02,-1.574661e-01,-2.309440e-02,-1.763875e-01,-2.116787e-01,3.470647e-02,6.027919e-02,-2.311456e-01,-2.437825e-01
75%,1.735590e-01,8.301332e-01,-7.942616e-03,8.156141e-01,-1.560231e-01,8.043581e-01,8.294525e-01,8.079834e-01,8.258958e-01,-7.898759e-02,...,1.807914e-01,8.099809e-01,-5.487777e-02,2.407781e-01,-6.886846e-02,-2.105475e-02,8.167297e-01,8.192050e-01,-1.104176e-01,-6.861488e-02
max,1.622165e+01,2.399673e+00,1.053503e+01,2.507723e+00,1.937210e+01,2.533230e+00,2.485380e+00,2.379807e+00,2.512349e+00,1.645942e+01,...,5.691483e+01,2.677082e+00,3.329260e+01,8.501020e+01,1.835081e+01,1.697913e+01,2.378367e+00,2.331103e+00,2.431996e+01,2.214102e+01


In [30]:
X_test.describe()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
count,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,...,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000,540000.000000
mean,0.080687,0.077645,-0.078056,-0.041544,0.081304,0.097733,0.040102,-0.059514,0.007312,-0.019910,...,0.012587,0.101328,-0.001065,-0.021900,-0.044187,-0.064901,-0.067551,0.032862,-0.001251,0.065453
std,1.083840,0.993237,0.899774,0.990365,1.121358,1.003550,1.014691,0.981529,0.992072,0.947492,...,0.847538,0.993147,0.996193,1.138171,0.909490,0.916864,1.005941,1.007891,0.987771,1.105185
min,-7.532133,-2.418083,-3.753202,-2.601741,-9.692036,-2.391948,-2.446307,-2.488669,-2.363317,-6.700183,...,-6.048436,-2.517356,-16.640739,-47.097506,-6.716204,-9.630058,-2.879125,-2.506219,-10.767813,-9.166884
25%,-0.503308,-0.753731,-0.506183,-0.802576,-0.357760,-0.740046,-0.809795,-0.834337,-0.803466,-0.272361,...,-0.401714,-0.697812,-0.247426,-0.331985,-0.269809,-0.411467,-0.917812,-0.789769,-0.322867,-0.382760
50%,-0.364895,0.102670,-0.346200,-0.065082,-0.253521,0.154031,0.123590,-0.074381,-0.034167,-0.181468,...,-0.104877,0.085895,-0.160082,-0.064072,-0.173164,-0.239575,-0.057358,0.098706,-0.218000,-0.230818
75%,0.289850,0.892694,-0.110918,0.754333,-0.125064,0.912623,0.884632,0.724549,0.830465,-0.084048,...,0.205613,0.917345,-0.060105,0.202076,-0.074844,-0.054329,0.763144,0.861130,-0.099253,-0.054339
max,16.002399,2.533101,10.496809,2.348723,18.533103,2.487874,2.574875,2.431848,2.375779,18.215110,...,16.826803,2.551455,34.818323,95.670397,18.070106,19.086215,2.350146,2.512489,24.342883,19.974776


In [35]:
y.describe()

count    600000.000000
mean          0.506010
std           0.499964
min           0.000000
25%           0.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: target, dtype: float64

## Modelo

In [33]:
### Definindo os callbacks
early_stopping = EarlyStopping(
    monitor='val_loss', 
    min_delta=0, 
    patience= 20, 
    verbose=0,
    mode='min', 
    baseline=None, 
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.3,
    patience=20,
    mode='min'
)

In [34]:
### Criando baseline-model
def get_model():
    inp = Input(shape=X.shape[1], name='input')
    h = Dense(128, activation='swish')(inp)
    h = Dropout(0.25)(h)
    h = Dense(64, activation='swish')(h)
    h = Dropout(0.25)(h)
    h = Dense(32, activation='swish')(h)
    h = Dropout(0.25)(h)
    h = Dense(1, activation='sigmoid')(h)
    
    model = Model(inputs=inp, outputs=h)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=1e-3),
        metrics=['AUC']
    )
    return model

In [37]:
EPOCHS = 135
BATCH_SIZE = 1024
VERBOSE = 0
N_SPLITS = 20

### Validação Cruzada
cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

scores = {fold:None for fold in range(cv.n_splits)}
predictions = []

for fold, (idx_train, idx_valid) in enumerate(cv.split(X, y)):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]

    model = get_model()

    print('**'*20)
    print(f"Fold {fold+1} || Training")
    print('**'*20)
    
    history = model.fit(
        X_train, y_train,
        validation_data=(X_valid, y_valid),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        verbose=VERBOSE,
        callbacks=[
            early_stopping,
            reduce_lr
        ]
    )
    
    scores[fold] = (history.history)
    
    print(f"Fold {fold+1} || Max Validation AUC: {np.max(scores[fold]['val_auc'])}")
    
    prediction = model.predict(X_test, batch_size=BATCH_SIZE).reshape(1,-1)[0]
    predictions.append(prediction)

print('**'*20)
print('Finished Training')
print('**'*20)

overall_auc = [np.max(scores[fold]['val_auc']) for fold in range(cv.n_splits)]
print('Overall Mean AUC: ', np.mean(overall_auc))

****************************************
Fold 1 || Training
****************************************
Fold 1 || Max Validation AUC: 0.753835916519165
****************************************
Fold 2 || Training
****************************************
Fold 2 || Max Validation AUC: 0.7546931505203247
****************************************
Fold 3 || Training
****************************************
Fold 3 || Max Validation AUC: 0.7525142431259155
****************************************
Fold 4 || Training
****************************************
Fold 4 || Max Validation AUC: 0.7545899152755737
****************************************
Fold 5 || Training
****************************************
Fold 5 || Max Validation AUC: 0.7547264099121094
****************************************
Fold 6 || Training
****************************************
Fold 6 || Max Validation AUC: 0.7575193643569946
****************************************
Fold 7 || Training
****************************************


In [39]:
sub_data.target = prediction
sub_data.head()

,id,target
0,600000,0.720930
1,600001,0.738561
2,600002,0.715054
3,600003,0.315898
4,600004,0.714447


In [40]:
sub_data.to_csv('submission.csv', index=False)